In [11]:
import numpy as np
from scipy.special import expit 
from sklearn.datasets import load_breast_cancer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split, cross_val_score

we chose to use breast cancer dataset in our expriments because of its various features. 
the dataset contains `30` features. if we were to brute force the search on these many feature, we would have to go through 2 to the power of 30 different combinations and that will take forever. 

In [12]:
data = load_breast_cancer()
x, y = data.data, data.target
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.33)

we first use `KNN` with only `1` neighbors to see how well the model does without any optimized feature selection algorithm. 

In [13]:
KNN = KNeighborsClassifier(n_neighbors=1)

In [34]:
KNN.fit(x_train, y_train) 
print(f'accuracy: {KNN.score(x_test, y_test)}'); 

accuracy: 0.9202127659574468


in the next cell we will use `PSO` to select a subset of features. as you can see the model has a performance boost of about `6%` !. 

In [28]:
model = modelWrapper(x, y) 
p = PSO(30, x.shape[1], model) 
features = p.optimize(num_iter=10) 

best result at iteration 0: 0.9632253711201079
best result at iteration 1: 0.9659244264507422
best result at iteration 2: 0.9632253711201079
best result at iteration 3: 0.9684885290148447
best result at iteration 4: 0.9684885290148447
best result at iteration 5: 0.9684885290148447
best result at iteration 6: 0.9684885290148447
best result at iteration 7: 0.9684885290148447
best result at iteration 8: 0.9684885290148447
best result at iteration 9: 0.9684885290148447
best result on the test set -> 0.9787234042553191


next we will test to see how the new `HPSOLS` algorithm does on our dataset. 
the results are disappointing. we expected the model to do much better than `PSO`. it didn't, 
and after hour of debugging i still don't know why. :(

In [35]:
p = HPSOLS(30, x.shape[1], model)
features = p.optimize(num_iter=10)

best result at iteration 0: 0.9475708502024291
best result at iteration 1: 0.9579622132253711
best result at iteration 2: 0.9579622132253711
best result at iteration 3: 0.9422402159244264
best result at iteration 4: 0.9449392712550606
best result at iteration 5: 0.9422402159244264
best result at iteration 6: 0.9449392712550606
best result at iteration 7: 0.9449392712550606
best result at iteration 8: 0.9422402159244264
best result at iteration 9: 0.9475708502024291
best result on the test set -> 0.9787234042553191


In [27]:
class modelWrapper():
    """
    modelWrapper is used to handle basic operations 
    related to the dataset. it uses a model to make evaluations
    which will be used by PSO algorithm to determine the best particle.
    """
    def __init__(self, x, y, model=None):
        self.model = model or KNeighborsClassifier(n_neighbors=5)
        self.x_train, self.x_test, \
        self.y_train, self.y_test = train_test_split(self.normalize(x), y, test_size=0.33)
        self.init_corr()

    def evaluate(self, features):
        """
        evaluate is uses the given features and 
        returns mean accuracy using k-fold cross validation
        """
        if np.all(features == 0) : 
            return 0 

        x_train = self.x_train[:, np.where(features)[0]]
        return cross_val_score(self.model,
                                x_train, 
                                self.y_train, 
                                cv=10,
                                n_jobs=-1, 
                                error_score=0).mean()

    def test(self, features) : 
        """
        test uses the given features to fit and evaluate the test set 
        """
        x_test = self.x_test[:, np.where(features)[0]]
        x_train = self.x_train[:, np.where(features)[0]]
        self.model.fit(x_train, self.y_train) 
        return self.model.score(x_test, self.y_test) 

    def init_corr(self):
        """
        init_corr first creates the correlation matrix, 
        it then initializes the correlations specified by the paper 
        """
        _, n = self.x_train.shape
        c = np.zeros(shape=(n, n))
            # TODO this can be replaced by a library function
        for i in range(n):
            for j in range(n):
                x, y = self.x_train[:, i], self.x_train[:, j]
                c[i][j] = np.sum((x - x.mean()) *
                                (y - y.mean()) / np.std(x) / np.std(y))
        self.corr = np.sum(c, axis=1) / (n - 1)
    
    def normalize(self, x) : 
        l , u = -1, 1
        x = l + (u - l) * (x - np.min(x, axis=0)) / (np.max(x, axis=0) - np.min(x, axis=0))
        return x 

class Particle:
    """
    particle class represents each particle in particle swarm
    """
    def __init__(self, dimension):
        self.position = np.random.randint(0, 2, size=(dimension))
        self.velocity = np.zeros(dimension)  
        self.memory = self.position


class PSO:
    """
    PSO is responsible for implementing basic particle swarm algorithms. 
    it is then subclassed by HPSOLS. 
    the code is very readable with minimal need for explanation.
    """
    def __init__(self,
                 num_particles=50,
                 dimension=30,
                 model=None):

        self.model = model
        self.dimension = dimension
        self.particles = [Particle(dimension) for _ in range(num_particles)]
        self.update_global()

    def optimize(self, num_iter=10, w=0.8, c1=2, c2=2):

        for i in range(num_iter):
            self.update_particles(w, c1, c2)
            print(f'best result at iteration {i}: {self.model.evaluate(self.best.position)}')
        print(f'best result on the test set -> {model.test(self.best.position)}')
        return self.best.position

    def update_particles(self, w, c1, c2):
        for p in self.particles:
            self.update_position(p, w, c1, c2)
            self.update_memory(p)
        self.update_global()

    def update_position(self, p, w, c1, c2):
        r1, r2 = np.random.rand(2)
        p.velocity = w * p.velocity + r1 * c1 * (p.memory - p.position) * \
            np.linalg.norm(p.velocity, 2) + r2 * c2 * (self.best.position - p.position)
        p.velocity = np.clip(p.velocity, -4, 4)
        p.position = (np.random.rand()
                      < expit(p.velocity)).astype(int)

    def update_memory(self, p):
        if self.model.evaluate(p.position) > self.model.evaluate(p.memory):
            p.memory = p.position

    def update_global(self):
        scores = [self.model.evaluate(p.position) for p in self.particles]
        self.best = self.particles[np.argmax(scores)]


class HPSOLS(PSO):
    """
    HPSOLS stands for `hybrid particle swarm optimization with local search`. 
    it a very fancy name for a search algorithm. HPSOLS inherits PSO and 
    implements the added functionalities specified by the paper. 
    """

    def __init__(self,
                 num_particles=50,
                 dimension=30,
                 model=None,
                 eps=None):

        super(HPSOLS, self).__init__(num_particles, dimension, model)
        self.init_num_features(eps)
        self.init_feature_set()

    def optimize(self, num_iter=10, alpha=0.65, w=0.8, c1=2, c2=2):
        for i in range(num_iter):
            self.update_particles(w, c1, c2)
            self.move_particles(alpha)
            print(f'best result at iteration {i}: {self.model.evaluate(self.best.position)}')
        print(f'best result on the test set -> {model.test(self.best.position)}')
        return self.best.position

    def move_particles(self, alpha):
        for p in self.particles:
            self.move_particle(p, alpha)

    def move_particle(self, p, alpha) :
        """
        move_particle performs the local search specified by the algorithm.
        """

        nd, ns = int(np.ceil(alpha * self.num_features)),\
                 int(np.floor((1 - alpha) * self.num_features))

        f = np.where(p.position) 
        fd = list(np.intersect1d(self.dissimilars, f))
        fs = list(np.intersect1d(self.similars, f))

        fd = self.add_features(nd, fd, self.dissimilars)
        fd = self.delete_features(nd, fd, self.dissimilars)

        fs = self.add_features(ns, fs, self.similars[::-1])
        fs = self.delete_features(ns, fs, self.similars[::-1])

        p.position = np.zeros(self.dimension)
        p.position[fs + fd] = 1 

    def add_features(self, n, f, s):
        i = len(s) - 1
        while len(f) < n : 
            while s[i] in f : 
                i -= 1 
            f.append(s[i])
        return f 

    def delete_features(self, n, f, s):
        i = 0
        while len(f) > n : 
            while s[i] not in f : 
                i += 1 
            f.remove(s[i]) 
        return f 

    def init_feature_set(self):
        c = np.argsort(self.model.corr)
        self.similars = c[len(c) // 2:]
        self.dissimilars = c[:len(c) // 2][::-1]

    def init_num_features(self, eps):

        eps = self.init_eps(eps)
        f = self.dimension
        k = int(eps * f)
        sf = np.arange(3, f)
        denom = np.cumsum(sf[::-1])
        prob = (f - sf) / denom[::-1]
        prob = prob[:k] / np.sum(prob[:k])
        self.num_features = np.random.choice(sf[:k], p=prob)

    def init_eps(self, eps):
        if not eps:
            eps = 0
            while eps < 0.15 or eps > 0.7:
                eps = np.random.rand(1)
        return eps
